In [643]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import paderbox as pb
from paderbox.array import interval
from padercontrib.database.fearless import Fearless
from padertorch import Model
from paderbox.transform import stft,fbank
from paderbox.transform import mfcc
import scipy

In [559]:
import torch
import torch.nn as nn
from torch import Tensor
from collections import OrderedDict 

In [560]:
import itertools
import padertorch as pt
import padercontrib as pc
import paderbox as pb

from pathlib import Path
from padertorch import Model
from tqdm import tqdm
from pprint import pprint

In [699]:
path = '/net/vol/saadmann/dump_audio_test/FS02_dev_001.wav'
rre_audio = np.round(pb.io.load_audio(path)).astype(bool)
rre_audio

array([False, False, False, ..., False, False, False])

In [700]:
pb.io.load_audio('/net/vol/saadmann/dump_audio_test/FS02_dev_001.wav')

array([0., 0., 0., ..., 0., 0., 0.])

In [701]:
intervals8k = interval.ArrayInterval(rre_audio)
intervals8k

ArrayInterval("1707680:1746960, 1952000:1971440, 2811040:3008000, 3014720:3022480, 3776000:3789200, 3793760:3847760, 4512000:4522480, 4529600:4590320, 4768000:4804720, 6611840:6625040, 6654560:6900880, 6903520:6945040, 7104000:7264000, 7633920:7684880, 7744000:7797840, 7807360:7808000, 10956640:11014160, 11341920:11353360, 11360000:11360720, 11368960:11402640, 12800000:12899920, 12983360:13067120, 13971520:14021680", shape=(14816000,))

In [698]:
len(intervals8k.intervals)

1

In [565]:
db = Fearless()
dataset_Dev_str = db.get_dataset('Dev_stream')
df_Dev_str = pd.DataFrame(dataset_Dev_str)

In [690]:
x_train_raw_audio = pb.io.load_audio(df_Dev_str['audio_path'][0]['observation'], 
                                                     start = intervals8k.intervals[1][0], 
                                                    stop = intervals8k.intervals[1][1], dtype=np.int16)
pb.io.play(x_train_raw_audio, sample_rate=8000)

## MFCC change

In [659]:
audio_int1=[]
for i in range(intervals8k.intervals[0][0],intervals8k.intervals[0][1], 100):
    audio_int1_slice = pb.io.load_audio(df_Dev_str['audio_path'][0]['observation'], 
                                                     start = i, frames=2000).astype(np.int64)
    print(np.mean(np.real(mfcc(audio_int1_slice))))

-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721353238
-16.25666721

In [660]:
dataset_Dev_str[0]['transcription'][0:4], dataset_Dev_str[0]['speaker_id'][0:4]

(['FLIGHT PROCEDURES.',
  'Go PROCEDURES.',
  'Upper clocks had C.S.M. SEP',
  'Rog.'],
 ['PROCEDURES1', 'FD1', 'PROCEDURES1', 'FD1'])

In [661]:
#To-do: pandas dataframe of Dev-set:
db = Fearless()
dataset_Dev_str = db.get_dataset('Dev_stream')
df_Dev_str = pd.DataFrame(dataset_Dev_str)
df_Dev_str.head()

,audio_path,end,num_samples,num_speakers,speaker_id,start,transcription,example_id,dataset
0,{'observation': '/net/db/fearless/Audio/Stream...,"[1716960, 1724240, 1738480, 1742720, 1967600, ...",14816000,8,"[PROCEDURES1, FD1, PROCEDURES1, FD1, CONTROL1,...","[1710480, 1716960, 1724720, 1739440, 1955360, ...","[FLIGHT PROCEDURES., Go PROCEDURES., Upper clo...",FS02_dev_001,Dev_stream
1,{'observation': '/net/db/fearless/Audio/Stream...,"[24800, 128560, 143040, 164080, 172320, 206240...",14400000,30,"[EECOM3, TRACK0, MADRID, TRACK0, EECOM3, MADRI...","[16800, 123360, 140320, 148560, 164080, 176000...","[SPAN EECOM conference., MADRID TRACK., MADRID...",FS02_dev_002,Dev_stream
2,{'observation': '/net/db/fearless/Audio/Stream...,"[98400, 158960, 169680, 262480, 526720, 767200...",14400000,26,"[CAPCOM1, EMU, FD1, BUZZ, BUZZ, BUZZ, NEIL, BU...","[36320, 98400, 162720, 169680, 285840, 544480,...","[Uh roger BUZZ, and break break COLUMBIA this ...",FS02_dev_003,Dev_stream
3,{'observation': '/net/db/fearless/Audio/Stream...,"[748080, 760480, 770000, 802720, 826080, 88120...",14955294,11,"[RETRO1, FIDO1, RETRO1, UNK, FIDO1, RETRO1, FI...","[739600, 757520, 761920, 797040, 815840, 84056...","[FIDO this is RETRO., Go ahead., Did you get t...",FS02_dev_004,Dev_stream
4,{'observation': '/net/db/fearless/Audio/Stream...,"[28240, 102000, 146320, 182160, 258720, 376480...",14400000,14,"[CAPCOM1, BUZZ, CAPCOM1, BUZZ, CAPCOM1, BUZZ, ...","[0, 61920, 103840, 172560, 190080, 337840, 390...",[Is HOUSTON uh radio check and verify T.V. cir...,FS02_dev_005,Dev_stream


In [662]:
len(df_Dev_str['speaker_id'][0])

62

In [663]:
print('speaker ID: ', df_Dev_str['speaker_id'][0][0], '\n',
      'transcription: ', df_Dev_str['transcription'][0][0], '\n',
      'start of sample: ', df_Dev_str['start'][0][0], '\n',
      'end of sample: ', df_Dev_str['end'][0][0], '\n',
      'number of samples: ', (df_Dev_str['end'][0][0]-df_Dev_str['start'][0][0]), '\n',
    'duration: ', (df_Dev_str['end'][0][0]-df_Dev_str['start'][0][0])/8000, 's')

speaker ID:  PROCEDURES1 
 transcription:  FLIGHT PROCEDURES. 
 start of sample:  1710480 
 end of sample:  1716960 
 number of samples:  6480 
 duration:  0.81 s


In [664]:
print('speaker ID: ', df_Dev_str['speaker_id'][0][3], '\n',
      'transcription: ', df_Dev_str['transcription'][0][3], '\n',
      'start of sample: ', df_Dev_str['start'][0][3], '\n',
      'end of sample: ', df_Dev_str['end'][0][3], '\n',
      'number of samples: ', (df_Dev_str['end'][0][3]-df_Dev_str['start'][0][3]), '\n',
    'duration: ', (df_Dev_str['end'][0][3]-df_Dev_str['start'][0][3])/8000, 's')

speaker ID:  FD1 
 transcription:  Rog. 
 start of sample:  1739440 
 end of sample:  1742720 
 number of samples:  3280 
 duration:  0.41 s


In [665]:
# TO-DO: speaker prediction using SAD sampling interval

In [666]:
def intervals(interval_num=0):
    return print('int1_start: ', intervals8k.intervals[interval_num][0], '\n',
                   'int1_end: ', intervals8k.intervals[interval_num][1], '\n',
                    'duration: ', (intervals8k.intervals[interval_num][1]-intervals8k.intervals[interval_num][0])/8000)
intervals(0)

int1_start:  1709760 
 int1_end:  1744880 
 duration:  4.39


In [673]:
audio_int1=[]
for i in range(intervals8k.intervals[1][0],intervals8k.intervals[1][1], 100):
    audio_int1_slice = pb.io.load_audio(df_Dev_str['audio_path'][0]['observation'], 
                                                     start = i, frames=2000).astype(np.float32)
    print(i, i+2000, i-(i+2000))
    audio_int1.append(audio_int1_slice)
#sample shift should be more than 

1952000 1954000 -2000
1952100 1954100 -2000
1952200 1954200 -2000
1952300 1954300 -2000
1952400 1954400 -2000
1952500 1954500 -2000
1952600 1954600 -2000
1952700 1954700 -2000
1952800 1954800 -2000
1952900 1954900 -2000
1953000 1955000 -2000
1953100 1955100 -2000
1953200 1955200 -2000
1953300 1955300 -2000
1953400 1955400 -2000
1953500 1955500 -2000
1953600 1955600 -2000
1953700 1955700 -2000
1953800 1955800 -2000
1953900 1955900 -2000
1954000 1956000 -2000
1954100 1956100 -2000
1954200 1956200 -2000
1954300 1956300 -2000
1954400 1956400 -2000
1954500 1956500 -2000
1954600 1956600 -2000
1954700 1956700 -2000
1954800 1956800 -2000
1954900 1956900 -2000
1955000 1957000 -2000
1955100 1957100 -2000
1955200 1957200 -2000
1955300 1957300 -2000
1955400 1957400 -2000
1955500 1957500 -2000
1955600 1957600 -2000
1955700 1957700 -2000
1955800 1957800 -2000
1955900 1957900 -2000
1956000 1958000 -2000
1956100 1958100 -2000
1956200 1958200 -2000
1956300 1958300 -2000
1956400 1958400 -2000
1956500 19

In [674]:
len(audio_int1_slice)

2000

In [675]:
for i in range(10):
    pb.io.play(audio_int1[i], sample_rate=8000, )

In [676]:
#exp_dir = '/net/vol/dheerajpr/models/SID/2021-08-06-15-22-56'
exp_dir = '/net/vol/dheerajpr/models/SID/2021-09-05-22-42-44'
ckpt_name = 'ckpt_best_loss.pth'
device = 0
model_SID = Model.from_storage_dir(
    exp_dir, consider_mpi=True, checkpoint_name=ckpt_name
)
model_SID.to(device)
model_SID.eval()

ConvAngularPen(
  size=ModelParameterSize(total_count=10721159, trainable_count=10721159, total_bytes=42884636, trainable_bytes=42884636)
  (convlayers): PT_ConvNet(
    size=ModelParameterSize(total_count=5322404, trainable_count=5322404, total_bytes=21289616, trainable_bytes=21289616)
    (layer1): Sequential(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01)
    )
    (layer2): Sequential(
      (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01)
      (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): LeakyReLU(negative_slope=0.01)
      (4): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (5): LeakyReLU(negative_slope=0.01)
      (6): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): LeakyReLU(negative_slope=0.01)
      (8): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1),

In [692]:
#speaker prediction for 1sample of 1st interval, 1 sample windowing, window length=0.5sec
sid_dict=dict()
fbank_data=[]

f_banks = fbank(x_train_raw_audio, sample_rate=8000, window_length=400, stft_shift=160, number_of_filters=64,
            stft_size=512,lowest_frequency=0,highest_frequency=None, preemphasis_factor=0.97,
            window=scipy.signal.windows.hamming, denoise=False)

fbank_data.append(f_banks)
float_fbank = np.float32(fbank_data)
float_fbank = np.squeeze(float_fbank,0)
float_fbank = np.expand_dims(float_fbank, axis=0)
float_fbank = np.expand_dims(float_fbank, axis=0)
float_fbank = torch.from_numpy(float_fbank).to(device)
sid_dict['features'] = (float_fbank)

out = model_SID(sid_dict)

In [693]:
torch.argmax(out['prediction'])

tensor(32, device='cuda:0')

In [691]:
#speaker prediction for all interval of 1st audio of Dev_stream, 
#1000 sample windowing, window length = 3100/8000 sec
preds=[]
with torch.no_grad():
    for j in tqdm(range(len(intervals8k.intervals))):
        for i in range(intervals8k.intervals[j][0],intervals8k.intervals[j][1], 1000):
            audio_int1_slice = pb.io.load_audio(df_Dev_str['audio_path'][0]['observation'], 
                                                             start = i, 
                                                            frames = 2000).astype(np.float32)
            sid_dict=dict()
            fbank_data=[]

            f_banks = fbank(audio_int1_slice, sample_rate=8000, window_length=400, stft_shift=160, number_of_filters=64,
                        stft_size=512,lowest_frequency=0,highest_frequency=None, preemphasis_factor=0.97,
                        window=scipy.signal.windows.hamming, denoise=False)
            f_banks = f_banks**-0.5
            fbank_data.append(f_banks)
            float_fbank = np.float32(fbank_data)
            float_fbank = np.squeeze(float_fbank,0)
            float_fbank = np.expand_dims(float_fbank, axis=0)
            float_fbank = np.expand_dims(float_fbank, axis=0)
            float_fbank = torch.from_numpy(float_fbank).to(device)
            sid_dict['features'] = (float_fbank)

            out = model_SID(sid_dict)
            preds.append(np.argmax(out['prediction'].data.cpu().numpy()))


100%|██████████| 34/34 [00:09<00:00,  3.77it/s]


In [581]:
dataset_Dev_str[0]['transcription'][0:4]

['FLIGHT PROCEDURES.', 'Go PROCEDURES.', 'Upper clocks had C.S.M. SEP', 'Rog.']

In [582]:
#speaker prediction for 1st interval, 1 sample windowing, window length=0.5sec
#speaker change detection can be performed from here, check for inconsistent prediction value.
#idea, when the window is crossing some overlapping region, the predictions should be random.
preds=[]
with torch.no_grad():
    for i in range(intervals8k.intervals[1][0],intervals8k.intervals[1][1], 100):
        audio_int1_slice = pb.io.load_audio(df_Dev_str['audio_path'][0]['observation'], 
                                                         start = i, 
                                                        frames = 4000, dtype=np.int16)#.astype(np.int16)
        sid_dict=dict()
        fbank_data=[]

        f_banks = fbank(audio_int1_slice, sample_rate=8000, window_length=400, stft_shift=160, number_of_filters=64,
                    stft_size=512,lowest_frequency=0,highest_frequency=None, preemphasis_factor=0.97,
                    window=scipy.signal.windows.hamming, denoise=False)
        #f_banks = f_banks**-0.5
        fbank_data.append(f_banks)
        float_fbank = np.float32(fbank_data)
        float_fbank = np.squeeze(float_fbank,0)
        float_fbank = np.expand_dims(float_fbank, axis=0)
        float_fbank = np.expand_dims(float_fbank, axis=0)
        float_fbank = torch.from_numpy(float_fbank).to(device)
        sid_dict['features'] = (float_fbank)

        out = model_SID(sid_dict)
        preds.append(np.argmax(out['prediction'].data.cpu().numpy()))


In [583]:
print(preds, len(preds))

[67, 67, 67, 67, 132, 56, 152, 132, 152, 2, 50, 2, 2, 49, 50, 50, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 49, 49, 49, 58, 58, 58, 32, 49, 32, 32, 144, 32, 32, 144, 144, 52, 15, 52, 52, 52, 52, 52, 15, 52, 52, 52, 52, 52, 52, 52, 52, 32, 52, 52, 52, 52, 52, 52, 113, 113, 113, 32, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 54, 56, 56, 56, 56, 56, 56, 56, 56, 56, 80, 80, 80, 80, 80, 80, 80, 80, 164, 164, 164, 15, 15, 15, 15, 15, 15, 15, 15, 164, 15, 15, 111, 164, 164, 111, 111] 171


In [584]:
import statistics
statistics.mode(preds) 

32

In [585]:
change = np.where(np.diff(preds,prepend=np.nan))[0]
print(change[:], len(change))

[  0   4   5   6   7   8   9  10  11  13  14  16  65  76  79  82  83  84
  86  87  89  91  92  93  98  99 107 108 114 117 118 134 135 144 152 155
 163 164 166 167 169] 41


In [586]:
#[1,1,1,1,1,2,2,2,2,2,4,4,4,4,4]->[1,1,1,1,1,0,0,0,0,0,1,1,1,1,1]
#  0-2000
#100-2100
#200-2200
#300-2300
#400-2400
#0:i*shift+frames
#1709760:i*100+frames+1709760
#stop = intervals8k.intervals[0][0]+2000+21*100

In [587]:
frames = 2000
sample_shift = 100
for i in range(len(change)):
    stop = intervals8k.intervals[0][0]+frames+change[i]*sample_shift
    predicted_change = pb.io.load_audio(df_Dev_str['audio_path'][0]['observation'], 
                                                         start = intervals8k.intervals[0][0], 
                                                         stop = stop).astype(np.float32)
    pb.io.play(predicted_change, sample_rate=8000)
    print(change[i])

0


4


5


6


7


...

163


164


166


167


169


In [588]:
pb.io.play(audio_int1[10], sample_rate=8000, )

In [589]:
#smoothing window

In [590]:
for idx, val in enumerate(preds):
    print(idx, val)

0 67
1 67
2 67
3 67
4 132
5 56
6 152
7 132
8 152
9 2
10 50
11 2
12 2
13 49
14 50
15 50
16 32
17 32
18 32
19 32
20 32
21 32
22 32
23 32
24 32
25 32
26 32
27 32
28 32
29 32
30 32
31 32
32 32
33 32
34 32
35 32
36 32
37 32
38 32
39 32
40 32
41 32
42 32
43 32
44 32
45 32
46 32
47 32
48 32
49 32
50 32
51 32
52 32
53 32
54 32
55 32
56 32
57 32
58 32
59 32
60 32
61 32
62 32
63 32
64 32
65 56
66 56
67 56
68 56
69 56
70 56
71 56
72 56
73 56
74 56
75 56
76 49
77 49
78 49
79 58
80 58
81 58
82 32
83 49
84 32
85 32
86 144
87 32
88 32
89 144
90 144
91 52
92 15
93 52
94 52
95 52
96 52
97 52
98 15
99 52
100 52
101 52
102 52
103 52
104 52
105 52
106 52
107 32
108 52
109 52
110 52
111 52
112 52
113 52
114 113
115 113
116 113
117 32
118 56
119 56
120 56
121 56
122 56
123 56
124 56
125 56
126 56
127 56
128 56
129 56
130 56
131 56
132 56
133 56
134 54
135 56
136 56
137 56
138 56
139 56
140 56
141 56
142 56
143 56
144 80
145 80
146 80
147 80
148 80
149 80
150 80
151 80
152 164
153 164
154 164
155 15
156 15
1

In [591]:
#TO-DO: do some training data prediction and see the scores with print(preds)

In [592]:
#To-do: pandas dataframe of Dev-set:
db = Fearless()
dataset_Train_str = db.get_dataset('Train_stream') #use train sid
dataset_Train_str_pd = pd.DataFrame(dataset_Train_str)
dataset_Train_str_pd.head()

,audio_path,end,num_samples,num_speakers,speaker_id,start,transcription,example_id,dataset
0,{'observation': '/net/db/fearless/Audio/Stream...,"[71040, 99920, 171120, 204320, 209840, 234160,...",14400000,45,"[HOUSTONCOMMTECH2, NETWORK3, HOUSTONCOMMTECH2,...","[0, 73280, 107680, 171120, 205280, 213600, 240...",[To it about three different Pass. He's gettin...,FS02_train_001,Train_stream
1,{'observation': '/net/db/fearless/Audio/Stream...,"[45760, 142560, 195120, 226240, 254080, 257279...",14400000,24,"[FD1, CAPCOM2, FD1, CAPCOM2, GUIDANCE4, FD1, G...","[19840, 61440, 143840, 200720, 248720, 254080,...",[Okay that must mean he's gone through the AGS...,FS02_train_002,Train_stream
2,{'observation': '/net/db/fearless/Audio/Stream...,"[8720, 25920, 35280, 41840, 43360, 51200, 6520...",14400000,20,"[FD1, CONTROL1, CAPCOM2, GUIDANCE4, FD1, CONTR...","[0, 23280, 27040, 35280, 41840, 44320, 51200, ...","[Improper switch position., Oh, He's trying to...",FS02_train_003,Train_stream
3,{'observation': '/net/db/fearless/Audio/Stream...,"[16800, 20800, 32640, 45760, 49680, 74960, 118...",14400000,47,"[MCCFILEspk1, CVTS, NETWORK3, FD1, RTC1, MCCFI...","[0, 16800, 20800, 38560, 47440, 49680, 74960, ...",[Thing to do is to just pull the recycle immed...,FS02_train_004,Train_stream
4,{'observation': '/net/db/fearless/Audio/Stream...,"[136880, 169840, 173600, 183520, 193280, 24624...",14816000,9,"[FD1, CONTROL1, FD1, CAPCOM2, CONTROL1, CONTRO...","[129360, 138560, 170640, 176800, 184400, 24000...","[Did you copy there CONTROL?, FLIGHT it's caus...",FS02_train_005,Train_stream


In [593]:
example = dataset_Train_str

In [594]:
intervals8k_train = db.get_activity(example[0])
intervals8k_train

ArrayInterval("0:71040, 73280:99920, 107680:204320, 205280:209840, 213600:234160, 240320:245200, 246320:280240, 281520:285520, 550400:558720, 968720:976960, 984480:990240, 1004480:1035599, 1038640:1045760, 1050640:1062080, 1080240:1088080, 1095120:1113920, 1115920:1125600, 1134640:1140240, 1145520:1148720, 1151760:1231120, 1239120:1243920, 1256880:1277680, 1311920:1324880, 1337440:1346800, 1348560:1375760, 1377280:1382560, 1383280:1389200, 1396000:1398960, 1402240:1419520, 1420640:1443760, 1612960:1618960, 1620960:1622960, 1629840:1659600, 1661120:1676720, 1678480:1681440, 1685440:1690960, 1694480:1697120, 1702720:1718480, 1728080:1731680, 1753600:1784720, 1800240:1823440, 1826960:1855200, 1856480:1864720, 1869840:1873680, 1874720:1883440, 1887440:1896800, 1945840:1964480, 1965760:1971680, 1974240:1980960, 1982080:2018720, 2023840:2029600, 2033600:2036880, 2039360:2046880, 2049840:2087120, 2090400:2115840, 2120160:2136480, 2136960:2146240, 2151280:2157520, 2168240:2175600, 2198400:2230

In [595]:
x_train_raw_audio = pb.io.load_audio(dataset_Train_str_pd['audio_path'][0]['observation'], 
                                                start = intervals8k_train.intervals[2][0], 
                                                stop = intervals8k_train.intervals[2][1]).astype(np.float32)
pb.io.play(x_train_raw_audio, sample_rate=8000)

In [596]:
dataset_Train_str_pd['transcription'][0][3], dataset_Train_str_pd['speaker_id'][0][3]

("Okay. But I guess the significant thing is here that he didn't hear the first two calls.",
 'NETWORK3')

In [597]:
(204320-107680)/8000

12.08

In [599]:
preds=[]
with torch.no_grad():
    for k in range(2000,30000, 1000):
        for i in range(intervals8k_train.intervals[2][0],intervals8k_train.intervals[2][1], 500):
            audio_int1_slice = pb.io.load_audio(dataset_Train_str_pd['audio_path'][0]['observation'], 
                                                             start = i, 
                                                            frames = k, dtype=np.int16)
            sid_dict=dict()
            fbank_data=[]

            f_banks = fbank(audio_int1_slice, sample_rate=8000, window_length=400, stft_shift=160, number_of_filters=64,
                        stft_size=512,lowest_frequency=0,highest_frequency=None, preemphasis_factor=0.97,
                        window=scipy.signal.windows.hamming, denoise=False)
            #f_banks = f_banks**-0.5
            fbank_data.append(f_banks)
            float_fbank = np.float32(fbank_data)
            float_fbank = np.squeeze(float_fbank,0)
            float_fbank = np.expand_dims(float_fbank, axis=0)
            float_fbank = np.expand_dims(float_fbank, axis=0)
            float_fbank = torch.from_numpy(float_fbank).to(device)
            sid_dict['features'] = (float_fbank)

            out = model_SID(sid_dict)
            preds.append(np.argmax(out['prediction'].data.cpu().numpy()))
        c = Counter(preds)
        b = [(i, c[i] / len(preds) * 100.0) for i, count in c.most_common()]
        print(max(set(preds), key = preds.count), b[0], k)

114 (114, 14.948453608247423) 2000
84 (84, 26.288659793814436) 3000
84 (84, 31.958762886597935) 4000
84 (84, 36.98453608247423) 5000
84 (84, 40.618556701030926) 6000
84 (84, 43.81443298969072) 7000
84 (84, 46.17083946980854) 8000
84 (84, 48.06701030927835) 9000
84 (84, 49.59908361970218) 10000
84 (84, 50.92783505154639) 11000
84 (84, 51.921274601686974) 12000
84 (84, 52.66323024054983) 13000
84 (84, 53.21173671689136) 14000
84 (84, 53.57142857142857) 15000
84 (84, 53.84879725085911) 16000
84 (84, 54.05927835051546) 17000
84 (84, 54.24499696785931) 18000
84 (84, 54.29553264604811) 19000
84 (84, 54.28648941942485) 20000
84 (84, 54.20103092783505) 21000
84 (84, 54.123711340206185) 22000
84 (84, 54.02999062792877) 23000
84 (84, 53.92200806813088) 24000
84 (84, 53.758591065292094) 25000
84 (84, 53.58762886597938) 26000
84 (84, 53.429817605075336) 27000
84 (84, 53.28369606720123) 28000
84 (84, 53.12960235640648) 29000


In [601]:
preds=[]
with torch.no_grad():
    for k in range(2000,30000, 1000):
        for i in range(intervals8k_train.intervals[0][0],intervals8k_train.intervals[0][1], 500):
            audio_int1_slice = pb.io.load_audio(dataset_Train_str_pd['audio_path'][0]['observation'], 
                                                             start = i, 
                                                            frames = k, dtype=np.int16)
            sid_dict=dict()
            fbank_data=[]

            f_banks = fbank(audio_int1_slice, sample_rate=8000, window_length=400, stft_shift=160, number_of_filters=64,
                        stft_size=512,lowest_frequency=0,highest_frequency=None, preemphasis_factor=0.97,
                        window=scipy.signal.windows.hamming, denoise=False)
            #f_banks = f_banks**-0.5
            fbank_data.append(f_banks)
            float_fbank = np.float32(fbank_data)
            float_fbank = np.squeeze(float_fbank,0)
            float_fbank = np.expand_dims(float_fbank, axis=0)
            float_fbank = np.expand_dims(float_fbank, axis=0)
            float_fbank = torch.from_numpy(float_fbank).to(device)
            sid_dict['features'] = (float_fbank)

            out = model_SID(sid_dict)
            preds.append(np.argmax(out['prediction'].data.cpu().numpy()))
        c = Counter(preds)
        b = [(i, c[i] / len(preds) * 100.0) for i, count in c.most_common()]
        print(max(set(preds), key = preds.count), b[0], k)

151 (84, 17.482517482517483) 2000
84 (84, 31.818181818181817) 3000
84 (84, 36.36363636363637) 4000
84 (84, 41.08391608391608) 5000
84 (84, 44.75524475524475) 6000
84 (84, 48.25174825174825) 7000
84 (84, 51.64835164835166) 8000
84 (84, 54.45804195804196) 9000
84 (84, 57.03185703185704) 10000
84 (84, 59.510489510489506) 11000
84 (84, 61.60203432930705) 12000
84 (84, 63.34498834498834) 13000
84 (84, 64.92738031199569) 14000
84 (84, 66.33366633366633) 15000
84 (84, 67.41258741258741) 16000
84 (84, 68.48776223776224) 17000
84 (84, 69.35417523652818) 18000
84 (84, 70.04662004662005) 19000
84 (84, 70.51895472948104) 20000
84 (84, 70.97902097902097) 21000
84 (84, 71.32867132867133) 22000
84 (84, 71.61474888747617) 23000
84 (84, 71.81514138035877) 24000
84 (84, 72.02797202797203) 25000
84 (84, 72.22377622377623) 26000
84 (84, 72.37762237762237) 27000
84 (84, 72.49417249417249) 28000
84 (84, 72.6023976023976) 29000


In [602]:
print(preds, len(preds))

[84, 84, 114, 134, 114, 114, 113, 88, 134, 88, 84, 113, 113, 11, 151, 151, 151, 151, 151, 151, 151, 151, 151, 151, 11, 88, 117, 84, 8, 57, 77, 57, 105, 77, 151, 151, 151, 151, 157, 56, 11, 77, 113, 34, 88, 84, 84, 84, 84, 114, 34, 157, 56, 56, 113, 157, 157, 84, 84, 84, 84, 77, 84, 117, 84, 84, 84, 117, 84, 111, 113, 113, 78, 57, 34, 84, 157, 157, 157, 84, 84, 157, 113, 88, 157, 84, 84, 84, 113, 84, 157, 157, 113, 113, 134, 110, 110, 151, 151, 151, 151, 151, 151, 88, 88, 114, 84, 114, 25, 77, 15, 151, 151, 11, 157, 60, 151, 157, 113, 88, 157, 157, 157, 157, 111, 111, 111, 15, 114, 23, 107, 105, 157, 72, 49, 0, 88, 66, 66, 66, 60, 151, 151, 84, 84, 84, 84, 84, 114, 88, 88, 84, 84, 84, 113, 113, 15, 56, 15, 15, 56, 56, 56, 56, 56, 77, 77, 84, 84, 8, 84, 84, 84, 84, 57, 105, 77, 56, 56, 157, 157, 157, 56, 157, 84, 97, 84, 84, 84, 84, 84, 117, 84, 84, 84, 56, 56, 157, 84, 84, 84, 84, 84, 84, 117, 117, 84, 84, 84, 84, 84, 84, 84, 84, 57, 57, 84, 84, 84, 156, 84, 84, 84, 84, 84, 157, 84, 84,

In [603]:
from collections import Counter
c = Counter(preds)
b = [(i, c[i] / len(preds) * 100.0) for i, count in c.most_common()]
print(max(set(preds), key = preds.count), b[0])

84 (84, 72.6023976023976)


In [604]:
change = np.where(np.diff(preds,prepend=np.nan))[0]
print(change[:], len(change))

[   0    2    3    4    6    7    8    9   10   11   13   14   24   25
   26   27   28   29   30   31   32   33   34   38   39   40   41   42
   43   44   45   49   50   51   52   54   55   57   61   62   63   64
   67   68   69   70   72   73   74   75   76   79   81   82   83   84
   85   88   89   90   92   94   95   97  103  105  106  107  108  109
  110  111  113  114  115  116  117  118  119  120  124  127  128  129
  130  131  132  133  134  135  136  137  140  141  143  148  149  151
  154  156  157  158  160  165  167  169  170  174  175  176  177  179
  182  183  184  185  186  191  192  195  197  198  204  206  214  216
  219  220  225  226  232  234  238  239  240  242  243  244  245  251
  252  253  254  255  256  258  260  262  265  266  267  268  269  270
  271  272  274  275  276  277  280  281  282  283  284  285  286  291
  292  293  297  299  300  301  305  307  308  309  311  312  317  318
  319  320  321  322  323  324  325  326  327  338  339  340  345  346
  347 

In [605]:
#To-Do: pass all the segments of a same speaker from train_segment and predict using model:

In [606]:
db = Fearless()
dataset_Train_seg = db.get_dataset('Dev_segment')
dataset_Train_seg_pd = pd.DataFrame(dataset_Train_seg)
dataset_Train_seg_pd.head()

,audio_path,num_samples,speaker_id,transcription,example_id,dataset
0,{'observation': '/net/db/fearless/Audio/Segmen...,37280,GNC1,ROG AND WED LIKE UH ZERO AND FOUR ONES [unk],FS02_dev_0001,Dev_segment
1,{'observation': '/net/db/fearless/Audio/Segmen...,56720,CAPCOM1,COLUMBIA THIS IS HOUSTON DID YOU COPY L O S A ...,FS02_dev_0002,Dev_segment
2,{'observation': '/net/db/fearless/Audio/Segmen...,49520,BUZZ,THE UH PANORAMA ILL BE TAKING IS ABOUT THIRTY ...,FS02_dev_0003,Dev_segment
3,{'observation': '/net/db/fearless/Audio/Segmen...,54880,GNC4,TEST WOULD BE CYCLING AT ONE COUNT PER FIVE SE...,FS02_dev_0004,Dev_segment
4,{'observation': '/net/db/fearless/Audio/Segmen...,47120,RETRO1,DID UH [unk] SOMETHING ABOUT P TWENTY DATA THA...,FS02_dev_0005,Dev_segment


In [607]:
#TO-do: prediction percentage per speaker: Train segments

In [610]:
speaker = pb.io.load_json(path = '/net/home/saadmann/saadImp/fearless/fearless/sid/labels_167.json')

In [621]:
speaker_list = speaker.keys()
speaker_list = [*speaker_list]

In [694]:
speaker_list

['AFD1',
 'AFD3',
 'AFD5',
 'AGC',
 'ALDS',
 'ALSEP',
 'ANTIGUA',
 'ARIA',
 'ASCENSION',
 'ASCENSIONCOMMTECH',
 'BERMUDA',
 'BOOSTER1',
 'BUZZ',
 'CALIFORNIA',
 'CANARY',
 'CAPCOM1',
 'CAPCOM2',
 'CARNARVON',
 'CCATSCOMMAND',
 'CCATSTELEMETRY',
 'CLTC',
 'COLLINS',
 'COMM',
 'COMMANAGER1',
 'COMMANAGER3',
 'COMMCONTROL1',
 'COMMCONTROL2',
 'COMMCONTROL3',
 'COMMCONTROL4',
 'COMPUTERMNO',
 'COMPUTERTM',
 'CONSUMABLES',
 'CONTROL1',
 'CONTROL2',
 'CPSPK',
 'CPSS',
 'CTSC',
 'CVTS',
 'DATACORE',
 'DEPUTYLOM',
 'DISPLAY',
 'ECS1',
 'ECS2',
 'EECOM1',
 'EECOM2',
 'EECOM3',
 'EECOM5',
 'EMU',
 'EPE',
 'EPS2',
 'EPS3',
 'EVA',
 'EXPERIMENTS',
 'FAO1',
 'FAO2',
 'FAO3',
 'FD1',
 'FD2',
 'FD3',
 'FEMALE2',
 'FIDO1',
 'FIDO3',
 'FORTYEIGHT',
 'GBM',
 'GCC',
 'GMIL',
 'GNC1',
 'GNC4',
 'GNN1',
 'GNN2',
 'GODDARDOPS',
 'GODDARDVOICE',
 'GOLDSTONE',
 'GOLDSTONECOMMTECH',
 'GOLDSTONEVIDEO',
 'GUAM',
 'GUAYMAS',
 'GUIDANCE1',
 'GUIDANCE2',
 'GUIDANCE3',
 'GUIDANCE4',
 'HAWAII',
 'HONEYSUCKLE',
 'HOUS

In [623]:
net5 = pd.DataFrame(dataset_Train_seg_pd.loc[dataset_Train_seg_pd['speaker_id']==speaker_list[0]])

In [624]:
net5.reset_index(drop=True, inplace=True)

In [625]:
net5['audio_path'][5]['observation']

'/net/db/fearless/Audio/Segments/ASR_track2/Dev/FS02_ASR_track2_dev_1516.wav'

In [626]:
net5['speaker_id'][0]

'AFD1'

In [627]:
with torch.no_grad():
    for j in range(len(speaker_list)):
        net5 = pd.DataFrame(dataset_Train_seg_pd.loc[dataset_Train_seg_pd['speaker_id']==speaker_list[j]])
        net5.reset_index(drop=True, inplace=True)
        preds=[]
        for i in range(len(net5)):
            audio_int1_slice = pb.io.load_audio(net5['audio_path'][i]['observation'],
                                                dtype=np.int16)#.astype(np.int16)
            sid_dict=dict()
            fbank_data=[]

            f_banks = fbank(audio_int1_slice, sample_rate=8000, window_length=400, stft_shift=160, number_of_filters=64,
                        stft_size=512,lowest_frequency=0,highest_frequency=None, preemphasis_factor=0.97,
                        window=scipy.signal.windows.hamming, denoise=False)
            #f_banks = f_banks**-0.5
            fbank_data.append(f_banks)
            float_fbank = np.float32(fbank_data)
            float_fbank = np.squeeze(float_fbank,0)
            float_fbank = np.expand_dims(float_fbank, axis=0)
            float_fbank = np.expand_dims(float_fbank, axis=0)
            float_fbank = torch.from_numpy(float_fbank).to(device)
            sid_dict['features'] = (float_fbank)

            out = model_SID(sid_dict)
            preds.append(np.argmax(out['prediction'].data.cpu().numpy()))
        c = Counter(preds)
        b = [(k, c[k] / len(preds) * 100.0) for k, count in c.most_common()]
        print('speaker: ',speaker_list[j],
              'prediction: ',speaker_list[max(set(preds), key = preds.count)],
              'percentage: ',b[0])


speaker:  AFD1 prediction:  AFD1 percentage:  (131, 11.76470588235294)
speaker:  AFD3 prediction:  HOUSTONCOMMTECH3 percentage:  (85, 100.0)
speaker:  AFD5 prediction:  AFD5 percentage:  (2, 45.45454545454545)
speaker:  AGC prediction:  AGC percentage:  (31, 50.0)
speaker:  ALDS prediction:  HOUSTONCOMMTECH1 percentage:  (83, 42.857142857142854)
speaker:  ALSEP prediction:  ALSEP percentage:  (5, 47.05882352941176)
speaker:  ANTIGUA prediction:  BUZZ percentage:  (12, 100.0)
speaker:  ARIA prediction:  ARIA percentage:  (7, 75.0)
speaker:  ASCENSION prediction:  ASCENSION percentage:  (8, 33.33333333333333)
speaker:  ASCENSIONCOMMTECH prediction:  ASCENSION percentage:  (8, 28.57142857142857)
speaker:  BERMUDA prediction:  HAWAII percentage:  (12, 50.0)
speaker:  BOOSTER1 prediction:  FD1 percentage:  (56, 35.714285714285715)
speaker:  BUZZ prediction:  BUZZ percentage:  (12, 87.29729729729729)
speaker:  CALIFORNIA prediction:  BUZZ percentage:  (12, 100.0)
speaker:  CANARY prediction:

speaker:  NETWORK2 prediction:  NETWORK2 percentage:  (113, 74.33628318584071)
speaker:  NETWORK3 prediction:  NETWORK3 percentage:  (114, 84.43579766536965)
speaker:  NETWORK4 prediction:  NETWORK5 percentage:  (116, 100.0)
speaker:  NETWORK5 prediction:  NETWORK5 percentage:  (116, 60.0)
speaker:  NETWORK6 prediction:  NETWORK6 percentage:  (117, 75.22935779816514)
speaker:  NETWORK7 prediction:  NETWORK7 percentage:  (118, 36.11111111111111)
speaker:  NETWORK9 prediction:  NETWORK7 percentage:  (118, 60.86956521739131)
speaker:  NIXON prediction:  NIXON percentage:  (120, 66.66666666666666)
speaker:  ONP prediction:  SURGEON2 percentage:  (149, 50.0)
speaker:  OPS prediction:  GOLDSTONE percentage:  (122, 18.75)
speaker:  PLSS1 prediction:  CAPCOM1 percentage:  (15, 100.0)
speaker:  PLSSCOMM prediction:  TRACK0 percentage:  (159, 53.84615384615385)
speaker:  PROCEDURES1 prediction:  PROCEDURES1 percentage:  (125, 22.58064516129032)
speaker:  PROCEDURES3 prediction:  RETRO3 percentag

In [628]:
#if for some dataset, the percentage is higher (>90%) 
#with a good match between prediction and original speaker,
#then a diarization task can be performed with good accuracy.

In [629]:
f_banks

array([[  291.15808564, 11322.11799334, 22582.48007396, ...,
         8975.20426559,  9429.84516261,  6860.55113562],
       [  367.7168967 , 11659.00533525, 23857.61373144, ...,
         4528.87769769,  3936.99760665,  8783.99037586],
       [  326.23551656, 11656.16738625, 23398.08905906, ...,
         7735.33610127,  5804.76917386, 14649.34832075],
       ...,
       [  274.26769885,  9889.82313472, 20152.82388647, ...,
        10083.11931481,  6850.49981045, 12908.015665  ],
       [  246.74705013,  9977.07901858, 20632.69283653, ...,
        17790.2330007 , 12752.32701654, 12827.50750483],
       [ 1133.89021202, 11857.43666231, 16787.98432133, ...,
         9483.22344045,  4573.96864679, 16950.80594869]])

In [630]:
print(preds, len(preds))

[166, 96, 83, 71, 112, 93, 157, 66, 163, 166, 154, 122, 166, 165, 71, 60, 105, 105, 113, 25, 105, 105, 122] 23


In [631]:
111 in preds

False

In [632]:
len(preds)

23

In [633]:
c = Counter(preds)
b = [(i, c[i] / len(preds) * 100.0) for i, count in c.most_common()]
print(max(set(preds), key = preds.count), b[0])

105 (105, 17.391304347826086)


In [634]:
#applying median filter
for i in range(1, 355, 2):
    median_preds = scipy.signal.medfilt(preds, i)[1:]
    change = np.where(np.diff(median_preds,prepend=np.nan))[0]
    print(len(change), 'window_len: ', i)

20 window_len:  1
11 window_len:  3
11 window_len:  5
12 window_len:  7
12 window_len:  9
10 window_len:  11
11 window_len:  13
14 window_len:  15
13 window_len:  17
12 window_len:  19
8 window_len:  21
8 window_len:  23
8 window_len:  25
10 window_len:  27
12 window_len:  29
11 window_len:  31
9 window_len:  33
7 window_len:  35
6 window_len:  37
6 window_len:  39
4 window_len:  41
2 window_len:  43
1 window_len:  45
1 window_len:  47
1 window_len:  49
1 window_len:  51
1 window_len:  53
1 window_len:  55
1 window_len:  57
1 window_len:  59
1 window_len:  61
1 window_len:  63
1 window_len:  65
1 window_len:  67
1 window_len:  69
1 window_len:  71
1 window_len:  73
1 window_len:  75
1 window_len:  77
1 window_len:  79
1 window_len:  81
1 window_len:  83
1 window_len:  85
1 window_len:  87
1 window_len:  89
1 window_len:  91
1 window_len:  93
1 window_len:  95
1 window_len:  97
1 window_len:  99
1 window_len:  101
1 window_len:  103
1 window_len:  105
1 window_len:  107
1 window_len:  1

In [635]:
change = np.where(np.diff(median_preds,prepend=np.nan))[0]
print(change[:], len(change))

[0] 1


In [636]:
list_preds = np.array([1,2,2,2,2,4,4,5,5,6,5,5])
scipy.signal.medfilt(list_preds, 3)[1:]

array([2., 2., 2., 2., 4., 4., 5., 5., 5., 5., 5.])

In [641]:
def strided_app(a, L, S ):  # Window len = L, Stride len/stepsize = S
    nrows = ((a.size-L)//S)+1
    n = a.strides[0]
    return np.lib.stride_tricks.as_strided(a, shape=(nrows,L), strides=(S*n,n))
for i in range(1, 170, 2):
    median_preds = np.median(strided_app(np.array(preds), i,1),axis=1)
    #print(median_preds)
    change = np.where(np.diff(median_preds,prepend=np.nan))[0]
    print(len(change), i)

21 1
11 3
10 5
9 7
9 9
6 11
3 13
5 15
5 17
3 19
1 21
1 23


ValueError: negative dimensions are not allowed

In [639]:
median_preds = np.median(strided_app(np.array(preds), 19,1),axis=1)
print(median_preds)
change = np.where(np.diff(median_preds,prepend=np.nan))[0]
print(len(change))

[112. 105. 105. 105. 112.]
3
